<a href="https://colab.research.google.com/github/Yagth/SemanticSearch/blob/main/SemanticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes
!pip install --upgrade langchain  -q
!pip install pinecone-client

In [2]:
!pip install sentence_transformers > /dev/null

In [ ]:
!git clone https://github.com/lm-sys/FastChat.git
%cd FastChat

In [ ]:
!pip3 install --upgrade pip  # enable PEP 660 support
!pip3 install -e .

In [1]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap
import requests
from bs4 import BeautifulSoup
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pinecone

model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Functions to scarp web data
Scap web data provided link from the user

In [ ]:
def get_html_content(url):
    response = requests.get(url)
    return response.content

In [ ]:
def get_plain_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    for script in soup(["script"]):
        script.extract()
    return soup.get_text()

In [ ]:
def split_text_into_chunks(plain_text, max_chars=2000):
    text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = max_chars,           # Usually chunk sizes are much larger than this
      chunk_overlap  = 20,        # Overlap is needed incase the text is split in odd places
      length_function = len,
    )
    chunks = text_splitter.split_text(plain_text)
    return chunks

In [ ]:
def scrape_text_from_url(url, max_chars=2000):
    html_content = get_html_content(url)
    plain_text = get_plain_text(html_content)
    text_chunks = split_text_into_chunks(plain_text, max_chars)
    return text_chunks

## Run this if you want to scrap web instead of load json

In [ ]:
url = input("Enter the URL to scrape text from: ")
plain_text_chunks = scrape_text_from_url(url)
print(plain_text_chunks)

# Functions to clean and extract data from a json file.


Installing gitpython and cloning the repo. that contains the exported data from slack

In [4]:
!pip install gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [5]:
import git
import os
import json
import datetime as dt

In [6]:
repo_url = "https://github.com/Yagth/MatterMost-LLM-test-Slack-export-Jun-19-2023---Jun-20-2023.git"
local_path = "/content/slackdata"

git.Repo.clone_from(repo_url, local_path)

<git.repo.base.Repo '/content/slackdata/.git'>

Pull updates from a repo.
Run this if you alread have clone the repo.

In [ ]:
repo = git.Repo('/content/slackdata/')
repo.git.pull()

'Updating 1e83421..84b977f\nFast-forward\n gptgenerated/{2023-06-30.json => 2023-06-19.json} | 0\n 1 file changed, 0 insertions(+), 0 deletions(-)\n rename gptgenerated/{2023-06-30.json => 2023-06-19.json} (100%)'

Functions that are used to load the json data of messages from the channels

In [34]:
def loadMetadata(filePath):
  with open(filePath, 'r') as file:
    jsonString = file.read()

  messages = json.loads(jsonString)
  metadatas = []

  for message in messages:
    timestamp = float(message['ts'])
    date = dt.datetime.fromtimestamp(timestamp)
    date = date.strftime("%Y-%m-%d")

    #If just to filter out channel joined messages which are irrelevant.
    if not ('subtype' in message):
      metadata = {
        'name':message['user_profile']['real_name'],
        'message':message['text'],
        'time': date,
        'channel':'gptgenerated',
        'ispublic':True
      }
      metadatas.append(metadata)

  return metadatas

def extractMessage(metadatas):
  messages = []
  for metadata in metadatas:
    messages.append(metadata['message'])

  return messages

In [40]:
metadatas = loadMetadata('/content/slackdata/gptgenerated/2023-06-19.json')
messages = extractMessage(metadatas)

[{'name': 'John Smith', 'message': "Do you know any alternative LLM models that can match GPT3's capabilities?", 'time': '2023-06-19', 'channel': 'gptgenerated', 'ispublic': True}, {'name': 'Alice Johnson', 'message': 'I heard OpenAI has released GPT4, which is supposed to be an improvement over GPT3.', 'time': '2023-06-19', 'channel': 'gptgenerated', 'ispublic': True}, {'name': 'Sarah Davis', 'message': "I've been using GPT2 for similar tasks, and it has been working well for me.", 'time': '2023-06-19', 'channel': 'gptgenerated', 'ispublic': True}, {'name': 'Michael Johnson', 'message': "I'm not familiar with LLM models, but I can look into it and provide some suggestions.", 'time': '2023-06-19', 'channel': 'gptgenerated', 'ispublic': True}, {'name': 'John Smith', 'message': 'GPT3 has been the go-to choice for most applications, but exploring alternatives is always a good idea.', 'time': '2023-06-19', 'channel': 'gptgenerated', 'ispublic': True}, {'name': 'Alice Johnson', 'message': "

# Vector Database setup
set up pinecone vector database and function to insert data along with a method to insert the datas in chunks

In [2]:
pinecone.init(api_key="API_KEY", environment="ENV")
index = pinecone.Index("INDEX-NAME")
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 41}},
 'total_vector_count': 41}

In [10]:
def addData(corpusData,url):
    id  = index.describe_index_stats()['total_vector_count']
    step = min(100, len(corpusData)) #The number of vectors to insert into the database at the same time
    records = []
    for i in range(len(corpusData)):
        chunk=corpusData[i]
        chunkInfo=(str(id+i),
                model.embed_query(chunk),
                {'title': url,'context': chunk})
        records.append(chunkInfo)
        if len(records) >= step:
          index.upsert(vectors=records)
          print('Batch no. ' + str(len(records)))
          records = []

def addMessage(messages, metadatas):
    id  = index.describe_index_stats()['total_vector_count']
    step = min(100, len(messages)) #The number of vectors to insert into the database at the same time
    records = []
    for i in range(len(messages)):
        chunk= messages[i]
        chunkInfo=(str(id+i),
                model.embed_query(chunk),
                metadatas[i])
        records.append(chunkInfo)

        if len(records) >= step:
          index.upsert(vectors=records)
          print('Batch no. ' + str(len(records)))
          records = []


## Run the following if you are scrapping web

In [ ]:
len(plain_text_chunks)
plain_text_chunks[10]

In [ ]:
addData(plain_text_chunks,url)
index.describe_index_stats()

## Run the following if you are using json file

In [41]:
addMessage(messages, metadatas)
index.describe_index_stats()

Batch no. 41


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 41}},
 'total_vector_count': 41}

In [3]:
def find_match(query,k):
    query_em = model.embed_query(query)
    result = index.query(
      query_em,
      top_k=k,
      include_values=True,
      include_metadata=True
    )

    return [result['matches'][i]['metadata'] for i in range(k)],[result['matches'][i]['metadata']['message'] for i in range(k)],[result['matches'][i]['score'] for i in range(k)]

def printResult(datas, metadatas, scores):
  for i in range(len(datas)):
    print("Entry: " + str(i))
    print("\tData: " + datas[i])
    print("\tScore: " + str(scores[i]))
    print("\tLink: " + str(metadatas[i]))


#Prompt
Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    return header + context + "\n\n" + query + "\n"

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
def create_prompt(context,query):
    # header = "Answer the question as truthfully as possible using only the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    prompt = '''
Give the most relevant response using only the context provided below.
Don't use any other context outside the given context. If the given context isn't enough to give a response, print 'The given context isn't enough to give a valid respone'

###Context:
    {context}
###Question:
    > {query}
###Response:
    '''.format(context=context, query=query)
    # return header +"\nContext:" + context + "\n\nQuestion: " + query + "\n"
    return prompt

def init_model():
      # Load the model
    print("Tokenizing model...")


    tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
    # tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.3")

    print("Creating model...")
    model = LLaMAForCausalLM.from_pretrained(
        "decapoda-research/llama-7b-hf",
        load_in_8bit=True,
        device_map="auto",
    )
    print("Fine tunning model...")
    model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")
    # model = AutoModelForCausalLM.from_pretrained(model, "lmsys/vicuna-7b-v1.3")
    # inputs = tokenizer(
        # prompt,
        # return_tensors="pt",
    # )

    return tokenizer, model

def generate_answer(prompt, tokenizer, model):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    print("Generating config model...")
    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    # for s in generation_output.sequences:
        # print(tokenizer.decode(s))
    return (tokenizer.decode(generation_output.sequences[0]))


In [6]:
tokenizer, modelNew = init_model()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Tokenizing model...
Creating model...


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

Fine tunning model...


In [ ]:
question = "What is decided about LLM Alternative?"
metadatas, datas, scores = find_match(question,5)
printResult(datas, metadatas, scores)

In [ ]:
combinedData = [[data, metadata] for data, metadata, score in zip(datas, metadatas, scores) if score > 0.2 and metadata['ispublic']]
context = "\n".join(""+data[0] + "\nBy-> "+data[1]['name'] +"  date-> "+str(data[1]['time'])+"\n" for data in combinedData)
if context == "":
  context = "Not enough context"
print(context)


In [ ]:
prompt = create_prompt(context, question)
response = generate_answer(prompt, tokenizer, modelNew)
lines = response.split("\n")
response = lines[len(lines) - 1].strip()
print(prompt)
print(response)

# Web appliation server for the model
This section involves creating a web server application using falsk that will host it externally using ngrok to be accessed globally.

In [ ]:
!pip install flask-ngrok

Installing ngrok

In [ ]:
!pip install pyngrok

The following cell is used to set up the ngrok with the environment variable need to run it. The xxxx is a place holder for the authentication token that can be found from the ngrok website

In [ ]:
!ngrok authtoken xxxx

In [7]:
from flask import Flask, request, render_template, jsonify
from flask_ngrok import run_with_ngrok

In [8]:
app = Flask(__name__)

run_with_ngrok(app)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/generateResponse',methods=['POST'])
def generateResponse():
    question = request.form['searchStr']

    metadatas, datas, scores = find_match(question,5)
    printResult(datas, metadatas, scores)

    combinedData = [[data, metadata] for data, metadata, score in zip(datas, metadatas, scores) if score > 0.2 and metadata['ispublic']]
    context = "\n".join(""+data[0] + "\nBy-> "+data[1]['name'] +"  date-> "+str(data[1]['time'])+"\n" for data in combinedData)
    if context == "":
      context = "Not enough context"

    prompt = create_prompt(context, question)
    response = generate_answer(prompt, tokenizer, modelNew)
    lines = response.split("\n")
    response = lines[len(lines) - 1].strip()

    data = {
        "response": response,
        "context": context,
    }

    return jsonify(data)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://a218-34-82-30-163.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Jul/2023 11:35:07] "GET / HTTP/1.1" 200 -


Entry: 0
	Data: I'm not familiar with LLM models, but I can look into it and provide some suggestions.
	Score: 0.725169599
	Link: {'channel': 'gptgenerated', 'ispublic': True, 'message': "I'm not familiar with LLM models, but I can look into it and provide some suggestions.", 'name': 'Michael Johnson', 'time': datetime.date(2023, 6, 19)}
Entry: 1
	Data: Do you know any alternative LLM models that can match GPT3's capabilities?
	Score: 0.503625393
	Link: {'channel': 'gptgenerated', 'ispublic': True, 'message': "Do you know any alternative LLM models that can match GPT3's capabilities?", 'name': 'John Smith', 'time': datetime.date(2023, 6, 19)}
Entry: 2
	Data: GPT3 is a powerful model, no doubt, but its size and computational requirements make it less practical for our semantic search application.
	Score: 0.111373752
	Link: {'channel': 'gptgenerated', 'ispublic': True, 'message': 'GPT3 is a powerful model, no doubt, but its size and computational requirements make it less practical for o

...

INFO:werkzeug:127.0.0.1 - - [01/Jul/2023 11:43:41] "POST /generateResponse HTTP/1.1" 200 -


Entry: 0
	Data: Has anyone tried EleutherAI's GPT-Neo? It's an open-source alternative to GPT3 and might be worth exploring.
	Score: 0.0778465569
	Link: {'channel': 'gptgenerated', 'ispublic': True, 'message': "Has anyone tried EleutherAI's GPT-Neo? It's an open-source alternative to GPT3 and might be worth exploring.", 'name': 'John Smith', 'time': datetime.date(2023, 6, 19)}
Entry: 1
	Data: Have you looked into CTRL? It's a conditional transformer language model that allows fine-grained control over the generated text.
	Score: 0.0676919
	Link: {'channel': 'gptgenerated', 'ispublic': True, 'message': "Have you looked into CTRL? It's a conditional transformer language model that allows fine-grained control over the generated text.", 'name': 'Alice Johnson', 'time': datetime.date(2023, 6, 19)}
Entry: 2
	Data: I'm not familiar with LLM models, but I can look into it and provide some suggestions.
	Score: 0.0596404299
	Link: {'channel': 'gptgenerated', 'ispublic': True, 'message': "I'm not

INFO:werkzeug:127.0.0.1 - - [01/Jul/2023 11:44:03] "POST /generateResponse HTTP/1.1" 200 -


Entry: 0
	Data: I'm not familiar with LLM models, but I can look into it and provide some suggestions.
	Score: 0.827054739
	Link: {'channel': 'gptgenerated', 'ispublic': True, 'message': "I'm not familiar with LLM models, but I can look into it and provide some suggestions.", 'name': 'Michael Johnson', 'time': datetime.date(2023, 6, 19)}
Entry: 1
	Data: Do you know any alternative LLM models that can match GPT3's capabilities?
	Score: 0.657356203
	Link: {'channel': 'gptgenerated', 'ispublic': True, 'message': "Do you know any alternative LLM models that can match GPT3's capabilities?", 'name': 'John Smith', 'time': datetime.date(2023, 6, 19)}
Entry: 2
	Data: What about XLNet? It's a transformer-based model that has shown promising results in various NLP benchmarks.
	Score: 0.277602
	Link: {'channel': 'gptgenerated', 'ispublic': True, 'message': "What about XLNet? It's a transformer-based model that has shown promising results in various NLP benchmarks.", 'name': 'Michael Johnson', 'tim

INFO:werkzeug:127.0.0.1 - - [01/Jul/2023 11:44:36] "POST /generateResponse HTTP/1.1" 200 -
